In [1]:
!pip install pandas fiona shapely pyproj rtree # à faire obligatoirement en premier pour utiliser rtree ou pygeos pour les jointures spatiales
!pip install geopandas
!pip install pandas-bokeh

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

In [3]:
import requests
import tempfile
import zipfile

temporary_location = tempfile.gettempdir()

def download_unzip(url, dirname = tempfile.gettempdir(), destname = "borders"):
  myfile = requests.get(url)
  open(dirname + '/' + destname + '.zip', 'wb').write(myfile.content)
  with zipfile.ZipFile(dirname + '/' + destname + '.zip', 'r') as zip_ref:
      zip_ref.extractall(dirname + '/' + destname)

In [4]:
url1 = "https://www.data.gouv.fr/fr/datasets/r/07b7c9a2-d1e2-4da6-9f20-01a7b72d4b12"
download_unzip(url1)
communes = gpd.read_file(temporary_location + "/borders/communes-20190101.json")

url2 = "https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr"
arrondissements = gpd.read_file(url2)

url3 = "https://www.data.gouv.fr/fr/datasets/r/322480e7-824b-402d-8dec-ebff968a161d"
camping = gpd.read_file(url3)

url4 = "https://www.data.gouv.fr/fr/datasets/r/6987b47d-7d97-4e3d-8508-ac924c48fc95"
hopitaux = gpd.read_file(url4)

url5 = "https://www.data.gouv.fr/fr/datasets/r/52b2a283-7171-426d-99e1-63efe071cdf0"
police = gpd.read_file(url5)

url6 = "https://www.data.gouv.fr/fr/datasets/r/058154d2-a462-48b7-b726-b77905d16be9"
mobile = gpd.read_file(url6)

url7 = "https://www.data.gouv.fr/fr/datasets/r/86ea48a0-dd94-4a23-b71c-80d3041d7db2"
defibrillateurs = gpd.read_file(url7)

In [5]:
arrondissements = arrondissements.rename(columns = {"c_arinsee" : "insee"})
arrondissements['insee'] = arrondissements['insee'].astype(str)
communes = communes[~communes.insee.str.startswith("75")].append(arrondissements)
communes["dep"] = communes.insee.str[:2]
#communes.head()

In [6]:
#camping.head()

In [7]:
#hopitaux.head()

In [8]:
#police.head()

In [9]:
#mobile.head()

In [10]:
idf_polygon = communes[communes['dep'].isin(['75', '92', '93', '94', '78', '91', '77', '95'])].geometry.unary_union
paris_polygon = communes[communes['dep'].isin(['75'])].geometry.unary_union
essonne_polygon = communes[communes['dep'].isin(['91'])].geometry.unary_union
yvelines_polygon = communes[communes['dep'].isin(['78'])].geometry.unary_union
valdoise_polygon = communes[communes['dep'].isin(['95'])].geometry.unary_union
seineetmarne_polygon = communes[communes['dep'].isin(['77'])].geometry.unary_union
hautsdeseine_polygon = communes[communes['dep'].isin(['92'])].geometry.unary_union
seinesaintdenis_polygon = communes[communes['dep'].isin(['93'])].geometry.unary_union
valdemarne_polygon = communes[communes['dep'].isin(['94'])].geometry.unary_union

idf_polygon_df = gpd.GeoDataFrame(geometry = [idf_polygon])
paris_polygon_df = gpd.GeoDataFrame(geometry = [paris_polygon])
essonne_polygon_df = gpd.GeoDataFrame(geometry = [essonne_polygon])
yvelines_polygon_df = gpd.GeoDataFrame(geometry = [yvelines_polygon])
valdoise_polygon_df = gpd.GeoDataFrame(geometry = [valdoise_polygon])
seineetmarne_polygon_df = gpd.GeoDataFrame(geometry = [seineetmarne_polygon])
hautsdeseine_polygon_df = gpd.GeoDataFrame(geometry = [hautsdeseine_polygon])
seinesaintdenis_polygon_df = gpd.GeoDataFrame(geometry = [seinesaintdenis_polygon])
valdemarne_polygon_df = gpd.GeoDataFrame(geometry = [valdemarne_polygon])

In [11]:
frames = [paris_polygon_df, essonne_polygon_df, valdemarne_polygon_df, seineetmarne_polygon_df, yvelines_polygon_df, seinesaintdenis_polygon_df, hautsdeseine_polygon_df, valdoise_polygon_df]
departements_idf = pd.concat(frames)

In [12]:
departements_idf.crs = {'init' :'epsg:4326'}
departements_idf = departements_idf.to_crs(epsg = 4326)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [13]:
camping["classement"] = camping["classement"].str[:1].astype(int)

In [14]:
surface_3etoiles = camping[camping["classement"] <= 3]["nombre_d_emplacements"]/100
surface_5etoiles = camping[camping["classement"] >= 4]["nombre_d_emplacements"]/80
camping["surface"] = pd.concat([surface_3etoiles, surface_5etoiles])

In [15]:
camping = camping.rename(columns = {"nom_commercial" : "Nom", "adresse" : "Adresse", "nombre_d_emplacements" : "Emplacements", "classement" : "Classement", "surface" : "Surface"})

In [16]:
camping.crs = {'init' :'epsg:4326'}
camping = camping.to_crs(epsg = 4326)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [17]:
hopitaux_idf = hopitaux[hopitaux["geometry"].within(idf_polygon)]
hopitaux_idf["geometry"] = hopitaux_idf["geometry"].centroid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [18]:
hopitaux_idf.crs = {'init' :'epsg:4326'}
hopitaux_idf = hopitaux_idf.to_crs(epsg = 4326)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [19]:
police = police.to_crs(epsg = 4326)

In [20]:
police_idf = police[police['geometry'].within(idf_polygon)]

In [21]:
police_idf.crs = {'init' :'epsg:4326'}
police_idf = police_idf.to_crs(epsg = 4326)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [22]:
mobile = mobile.to_crs(epsg = 4326)

In [23]:
mobile_idf = mobile[mobile["dept"].isin(["75", "77", "78", "91", "92", "93", "94", "95"])]

In [24]:
mobile_polygon = mobile_idf.geometry.unary_union
sans_mobile_idf = idf_polygon.difference(mobile_polygon)
sans_mobile_idf_df = gpd.GeoDataFrame(geometry = [sans_mobile_idf])

In [25]:
sans_mobile_idf_df.crs = {'init' :'epsg:4326'}
sans_mobile_idf_df = sans_mobile_idf_df.to_crs(epsg = 4326)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [26]:
defibrillateurs = defibrillateurs.to_crs(epsg = 4326)

In [27]:
defibrillateurs_idf = defibrillateurs[defibrillateurs["geometry"].within(idf_polygon)]
defibrillateurs_idf["geometry"] = defibrillateurs_idf["geometry"].centroid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [28]:
defibrillateurs_idf.crs = {'init' :'epsg:4326'}
defibrillateurs_idf = defibrillateurs_idf.to_crs(epsg = 4326)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [29]:
m = communes[communes['dep'].isin(['75', '92', '93', '94', '78', '91', '77', '95'])].plot_bokeh(line_width = 0.1, fill_alpha = 0, legend = "Communes", title = "Sites de Camping en Ile-de-France", show_figure = False)
m.width = 1000
m.height = 700
departements_idf.plot_bokeh(figure = m, line_color = "black", fill_alpha = 0.3, color = "blue", legend = "Départements", show_figure = False)
sans_mobile_idf_df.plot_bokeh(figure = m, legend = "Pas de Couverture Mobile", color = "grey", line_width = 0.5, show_figure = False)
defibrillateurs_idf.plot_bokeh(figure = m, legend = "Défibrillateurs", size = 0.5, color = "green", show_figure = False)
police_idf.plot_bokeh(figure = m, color = "blue", size = 2, legend = "Police/Gendarmerie", hovertool_columns = ["official_name"], show_figure = False)
hopitaux_idf.plot_bokeh(figure = m, color = "yellow", size = 2, legend = "Hôpitaux/Cliniques",hovertool_columns = ["name"], show_figure = False)
camping.plot_bokeh(figure = m, size = 5, legend = "Campings (classés)", category = "Classement", colormap = "Reds", hovertool_columns = ["Nom", "Adresse", "Emplacements", "Classement", "Surface"])

Output hidden; open in https://colab.research.google.com to view.

In [30]:
#visualiser les défibrillateurs a moins de 50m des sites de camping?
#visualiser les sites sans couverture mobile?
#footnote :ecrire ce que veut dire les etoiles des sites veulent dire (connexion Internet,  permanence 24h/24) en bas du graphique ou en haut

In [31]:
communes_idf = communes[communes['dep'].isin(['75', '92', '93', '94', '78', '91', '77', '95'])]
communes_fusion = communes_idf[["insee", "nom", "surf_ha", "geometry", "dep"]]
communes_fusion["type"] = "communes"
communes_fusion = communes_fusion.rename(columns = {"nom" : "Nom", "surf_ha" : "Surface"})
communes_fusion.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,insee,Nom,Surface,geometry,dep,type
3981,91556,Saint-Hilaire,681.0,"POLYGON ((2.06320 48.44342, 2.07049 48.44691, ...",91,communes
3997,91148,Chauffour-lès-Étréchy,481.0,"POLYGON ((2.15000 48.50944, 2.15287 48.50948, ...",91,communes
3998,77395,Rumont,660.0,"POLYGON ((2.47361 48.27058, 2.47758 48.27101, ...",77,communes
3999,77003,Amponville,1193.0,"POLYGON ((2.50261 48.28749, 2.50728 48.28815, ...",77,communes
4000,77041,Boissy-aux-Cailles,1638.0,"POLYGON ((2.45610 48.32177, 2.45994 48.32128, ...",77,communes


In [32]:
departements_fusion = departements_idf
departements_fusion["type"] = "departements"
departements_fusion.head()

,geometry,type
0,"POLYGON ((2.34308 48.81609, 2.34171 48.81634, ...",departements
0,"POLYGON ((1.98086 48.31874, 1.97431 48.32335, ...",departements
0,"POLYGON ((2.57177 48.69199, 2.56613 48.69180, ...",departements
0,"POLYGON ((2.46743 48.14825, 2.47460 48.14900, ...",departements
0,"POLYGON ((1.86401 48.44458, 1.86104 48.44440, ...",departements


In [33]:
camping_fusion = camping[["Adresse", "dep", "Emplacements", "Nom", "Classement", "commune", "code_postal", "geometry", "Surface"]]
camping_fusion["type"] = "camping"
camping_fusion = camping_fusion.rename(columns = {"Adresse" : "Adresse_full"})
camping_fusion.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Adresse_full,dep,Emplacements,Nom,Classement,commune,code_postal,geometry,Surface,type
0,Route du Château d'eau,78,180,CAMPING HUTTOPIA RAMBOUILLET,3,RAMBOUILLET,78120,POINT (1.83569 48.63006),1.800,camping
1,11 CHEMIN DU CAMPING,77,440,CAMPING LES PRÉS DE LA FONTAINE,3,HERMÉ,77114,POINT (3.33855 48.47278),4.400,camping
2,26 Route de Vauvert,91,230,CARAVANING LE VAUVERT,4,ORMOY-LA-RIVIÈRE,91150,POINT (2.14394 48.41128),2.875,camping
3,chemin des ponceaux,77,140,CAMPING LES ACACIAS,2,BETON-BAZOCHES,77320,POINT (3.23946 48.70700),1.400,camping
4,24 Place de la gare de Faremoutiers,77,316,LE CHÊNE GRIS,4,POMMEUSE,77515,POINT (2.99998 48.80051),3.950,camping


In [35]:
hopitaux_fusion = hopitaux_idf[["name", "geometry", "addr:housenumber", "addr:street", "addr:postcode", "addr:city"]]
hopitaux_fusion["type"] = "hopitaux"
hopitaux_fusion = hopitaux_fusion.rename(columns = {"addr:housenumber" : "numero_hab", "addr:street" : "rue", "addr:postcode" : "code_postal", "addr:city" : "ville", "name" : "Nom"})
hopitaux_fusion.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Nom,geometry,numero_hab,rue,code_postal,ville,type
4,Institut hospitalier franco-britannique - Site...,POINT (2.27938 48.89160),None,None,None,None,hopitaux
9,Hôpital privé de la Seine-Saint-Denis,POINT (2.45695 48.93644),None,None,None,None,hopitaux
10,Hôpital Henri Mondor,POINT (2.45264 48.79595),None,None,None,None,hopitaux
19,Institut de Cardiologie,POINT (2.36702 48.83529),None,None,None,None,hopitaux
23,Hôpital privé Nord-Parisien,POINT (2.37743 48.98060),None,None,None,None,hopitaux


In [36]:
police_fusion = police_idf[["official_name", "police-FR", "osm_original_geom", "geometry", "addr-housenumber", "addr-street", "addr-postcode", "addr-city", "addr-full"]]
police_fusion["type"] = "police"
police_fusion = police_fusion.rename(columns = {"official_name" : "Nom", "addr-housenumber" : "numero_hab", "addr-street" : "rue", "addr-postcode" : "code_postal", "addr-city" : "ville", "addr-full" : "Adresse_full"})
police_fusion.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Nom,police-FR,osm_original_geom,geometry,numero_hab,rue,code_postal,ville,Adresse_full,type
3,Commissariat de police de Pantin,police,,POINT (2.40559 48.89318),14,Rue Eugène et Marie-Louise Cornet,93500,Pantin,None,police
65,None,police_municipale,,POINT (2.21544 48.75419),None,None,None,None,None,police
68,Commissariat de police de Guyancourt,police,SRID=3857;POLYGON((228220.151236704 6238526.89...,POINT (2.05030 48.78546),1,Rue de la Division Leclerc,78280,Guyancourt,None,police
69,Commissariat de police de Villeparisis,police,,POINT (2.60819 48.94469),None,None,None,None,None,police
71,Commissariat de police de Massy,police,SRID=3857;POLYGON((252944.644287904 6229275.46...,POINT (2.27246 48.73051),None,Rue de la Division Leclerc,91300,Massy,None,police


In [37]:
mobile_fusion = mobile_idf[["techno", "usage", "dept", "geometry"]]
mobile_fusion["type"] = "couverture mobile"
mobile_fusion = mobile_fusion.rename(columns = {"dept" : "dep"})
mobile_fusion.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,techno,usage,dep,geometry,type
75,4G,data,75,"POLYGON ((2.41634 48.84923, 2.41635 48.84870, ...",couverture mobile
77,4G,data,77,"MULTIPOLYGON (((3.07196 49.11776, 3.07246 49.1...",couverture mobile
78,4G,data,78,"MULTIPOLYGON (((1.60875 49.07771, 1.61083 49.0...",couverture mobile
91,4G,data,91,"POLYGON ((2.32072 48.74876, 2.32267 48.74812, ...",couverture mobile
92,4G,data,92,"POLYGON ((2.31989 48.90046, 2.31852 48.89964, ...",couverture mobile


In [38]:
sans_mobile_idf_df["type"] ="sans couverture mobile"
sans_mobile_idf_df

,geometry,type
0,"MULTIPOLYGON (((2.19227 49.17366, 2.19550 49.1...",sans couverture mobile


In [39]:
defibrillateurs_fusion = defibrillateurs_idf[["c_adr_num", "c_adr_voie", "c_com_cp", "c_com_insee", "c_com_nom", "geometry"]]
defibrillateurs_fusion["type"] = "defibrillateurs"
defibrillateurs_fusion = defibrillateurs_fusion.rename(columns = {"c_adr_num" : "numero_hab", "c_adr_voie" : "rue", "c_com_cp" : "code_postal", "c_com_nom" : "commune", "c_com_insee" : "insee"})
defibrillateurs_fusion.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,numero_hab,rue,code_postal,insee,commune,geometry,type
599,2,Avenue Sadi Carnot,91160,91044,Ballainvilliers,POINT (2.27780 48.67144),defibrillateurs
612,1,1 Rue de la Desserte Industrielle,91220,91103,Brétigny-sur-Orge,POINT (2.30031 48.59040),defibrillateurs
613,280,RN 6 Lieu-dit La Fontaine Ronde,77240,77495,Vert-Saint-Denis,POINT (2.61643 48.57777),defibrillateurs
615,9,Rue Jean Monnet,77410,77118,Claye-Souilly,POINT (2.66227 48.95279),defibrillateurs
616,3 a,6-8 Route de Cannes,77130,77482,Varennes-sur-Seine,POINT (2.95869 48.37390),defibrillateurs


In [40]:
frames_fusion = [communes_fusion, departements_fusion, camping_fusion, hopitaux_fusion, police_fusion, mobile_fusion, sans_mobile_idf_df, defibrillateurs_fusion]
base_fusion = pd.concat(frames_fusion)

In [41]:
base_fusion.head()

,insee,Nom,Surface,geometry,dep,type,Adresse_full,Emplacements,Classement,commune,code_postal,numero_hab,rue,ville,police-FR,osm_original_geom,techno,usage
3981,91556,Saint-Hilaire,681.0,"POLYGON ((2.06320 48.44342, 2.07049 48.44691, ...",91,communes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997,91148,Chauffour-lès-Étréchy,481.0,"POLYGON ((2.15000 48.50944, 2.15287 48.50948, ...",91,communes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3998,77395,Rumont,660.0,"POLYGON ((2.47361 48.27058, 2.47758 48.27101, ...",77,communes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3999,77003,Amponville,1193.0,"POLYGON ((2.50261 48.28749, 2.50728 48.28815, ...",77,communes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4000,77041,Boissy-aux-Cailles,1638.0,"POLYGON ((2.45610 48.32177, 2.45994 48.32128, ...",77,communes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
police_fusion.to_csv("police.csv")
camping_fusion.to_csv("camping.csv")
defibrillateurs_fusion.to_csv("defibrillateur.csv")